In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
IMG_HEIGHT = 28
IMG_WIDTH = 28

# Fashion MNIST dataset - 28x28 greyscale images. 60,000 images for training. 10,000 for testing. 10 classes
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

# Normalise pixel values to be between 0 and 1
train_images, test_images = train_images / 255, test_images / 255

class_names = ['t-shirt', 'trouser', 'jumper', 'dress', 'coat',
               'sandal', 'shirt', 'trainer', 'bag', 'boot']

print(f'Train images: {train_images.shape}\tTrain lables: {len(train_labels)}')
print(f'Test images: {test_images.shape}\tTest lables: {len(test_labels)}')

In [ ]:
# Verify the data by plotting the first few images in the dataset
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([]), plt.yticks([])
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

In [ ]:
# Build the neural network, specifying each of the layers
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

In [ ]:
# Specify the algorithm for backpropagation, the loss function and a performace metric
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Stop training early if the validation loss is constant or increasing for more than 3 epochs
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Train the network
history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels), callbacks=callback)

In [ ]:
def plot_metric(history, metric='loss'):
    plt.title(metric.capitalize())
    plt.plot(history.history[metric])
    plt.plot(history.history[f'val_{metric}'])
    plt.xlabel('Epoch'), plt.ylabel(metric.capitalize())
    plt.legend(['Training', 'Validation'])
    plt.show()

# Evaluate the network
plot_metric(history, 'loss')
plot_metric(history, 'accuracy')

# Test the network on unseen data
loss, acc = model.evaluate(test_images,  test_labels)

In [ ]:
# Predict the label for each image in the test set
predictions = model.predict(test_images)

In [ ]:
 def plot_image(predictions, label, img):
    plt.xticks([]), plt.yticks([])
    plt.imshow(img, cmap=plt.cm.binary)
    predicted_label = np.argmax(predictions)
    colour = 'blue' if predicted_label == label else 'red'
    plt.xlabel(f'{class_names[predicted_label]} {100 * np.max(predictions):2.0f}% ({class_names[label]})', color=colour)

def plot_value_array(predictions, label):
    plt.xticks(range(len(class_names))), plt.yticks([])
    plot = plt.bar(range(len(class_names)), predictions, color='gray')
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions)
    plot[predicted_label].set_color('red')
    plot[label].set_color('blue')

In [ ]:
# Plot the first test images, their predicted labels, and the true labels
# Correct predictions are blue and incorrect predictions are red
num_rows = 5
num_cols = 3
num_images = num_rows * num_cols

plt.figure(figsize=(2 * 2 * num_cols, 2 * num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 1)
    plot_image(predictions[i], test_labels[i], test_images[i])
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 2)
    plot_value_array(predictions[i], test_labels[i])
plt.show()

In [ ]:
def process_image(url):
    image = plt.imread(tf.keras.utils.get_file(origin=url))
    # Convert image to grayscale and invert colours
    image = tf.image.rgb_to_grayscale(tf.bitwise.invert(image))
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH]) / 255
    image = np.expand_dims(image, 0)
    return image

image = process_image('https://upload.wikimedia.org/wikipedia/commons/2/24/Blue_Tshirt.jpg')
prediction = model.predict(image)

plt.xticks([]), plt.yticks([])
plt.xlabel(f'{class_names[np.argmax(prediction)]} ({np.max(prediction):.2f})')
plt.imshow(image[0,:,:,:], cmap=plt.cm.binary)

In [ ]:
# Save the model
model.save('../saved/image-classifier-mnist.h5')